Veri analiz sürecinde bize yardımcı olacak kütüphanelerimizi import ederek işe başlayalım.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import missingno as mno

from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, recall_score, f1_score, precision_score

from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px

from sklearn import preprocessing

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Kütüphanelerimizi import ettik. Şimdi de kullanıcağımız veri çerçevesini ekleyelim.

In [ ]:
dfMain=pd.read_csv("../input/tabletpc-priceclassification/tablet.csv")
df=dfMain.copy()

İlk 5 gözlemi inceleyelim.

In [ ]:
df.head()

Son 5 gözlemi inceleyelim.

In [ ]:
df.tail()

Rastgele olacak şekilde 5 gözlemi inceleyelim.

In [ ]:
df.sample(5)

Veri setinin değişken ve gözlem sayısını öğrenelim

In [ ]:
df.shape

Özniteliklerimizi inceleyelim.

In [ ]:
df.info()

Eksik değer olup olmadığına bakalım.

In [ ]:
df.isna().sum()

'OnKameraMP' ve 'RAM' özniteliklerinde eksik veri olduğunu söyleyebilriz.

Sayısal değişkenlerin istatistik bilgilerini inceleyelim.

In [ ]:
df.describe().T

İstatistikleri inceleyip yorumlarda bulunalım.

* Ram Özniteliğinin standart sapmasının oldukça yüksek olduğunu, gözlemlerde ram'in fazla değişkenlik gösterdiğini söyleyebiliriz.

* Ram ve OnKameraMP özniteliklerinde eksik gözlem olduğunu söyleyebiliriz.

* OnKameraMP ve ArkaKameraMP özniteliklerinin bazı gözlemlerinde 0 olduğunu, bu gözlemlerde On Kamera ya da Arka Kamera bulunmadığını söyleyebiliriz.

Eksik verilerimizi düzenlemeye başlamadan önce missingno kütüphanesini kullanarak aralarındaki ilişkiyi inceleyelim.

In [ ]:
mno.matrix(df, figsize = (20, 6))
plt.show()

Eksik verileri doldurmak için aynı klasmandaki ürünlerin özelleiklerine bakalım.

In [ ]:
BosDegerler = df[df.isna().any(axis=1)]
BosDegerler

Eksik verileri doldurmak için ilişkilerini incelemek için ısı harisatını oluşturalım.

In [ ]:
corr = df.corr()
f,ax=plt.subplots(figsize=(25,15))
sns.heatmap(corr, annot=True, ax=ax)
plt.show()

Isı haritasını incelediğimizde ön kamera va arka kamera değerleri arasında güçlü ve pozitif bir ilişki olduğunu görebiliriz.

In [ ]:
plt.figure(figsize=(18,5))
ax = sns.swarmplot(x="ArkaKameraMP", y="OnKameraMP", data=df)


In [ ]:
data1=go.Violin(y=df.OnKameraMP,x=df.FiyatAraligi, box_visible=True, line_color='black',
                               meanline_visible=True, fillcolor='lightseagreen', opacity=0.6,
                               x0='Ön Kamera')
layout=dict(title='Ön Kamera İstatistikleri',
           xaxis=dict(title='Fiyat Aralığı',ticklen=5,zeroline=False)
           )
fig = go.Figure(data=data1,layout=layout)
iplot(fig)

Görüldüğü üzere fiyat aralığıyla OnKameraMP özniteliğinin bir ilişkisi bulunmamaktadır.

OnKameraMP özniteliğindeki boş gözlemleri, bulunduğu noktadaki ArkaKameraMP değerinden yola çıkarak dolduralım.

In [ ]:
df[df.OnKameraMP.isna()==True]

In [ ]:
ArkaKameraDegerleri=list(df[df.OnKameraMP.isna()==True].ArkaKameraMP)
ArkaKameraDegerleri

In [ ]:
for each in ArkaKameraDegerleri:
    
    OnKameraDegeri = (df[df.ArkaKameraMP==each].OnKameraMP).median()
    
    df.loc[(df['ArkaKameraMP'] == each)&(df.OnKameraMP.isna()), 'OnKameraMP'] = OnKameraDegeri

In [ ]:
df.isna().sum()

RAM özniteliğini doldurmak için ilk olarak fiyat aralığına göre değişimini inceleyelim.

In [ ]:
data2=go.Violin(y=df.RAM,x=df.FiyatAraligi, box_visible=True, line_color='black',
                               meanline_visible=True, fillcolor='lightseagreen', opacity=0.6,
                               x0='Total Bill')
layout=dict(title='RAM İstatistikleri',
           xaxis=dict(title='Fiyat Aralığı',ticklen=5,zeroline=False)
           )
fig = go.Figure(data=data2,layout=layout)
iplot(fig)

RAM özniteliğini doldururken fiyat aralığındaki benzerlerini inceleyerek doldurabiliriz.

In [ ]:
df[df.RAM.isna()==True].FiyatAraligi.unique()

Görüldüğü üzere eksik verilerimizin hepsi pahalı kategorisindedir. Eksik verileri bu kategorideki diğer gözlemlerin medyanıyla dolduralım.

In [ ]:
df[df.RAM.isna()==True]

In [ ]:
df.RAM.fillna(round(df[df.FiyatAraligi=="Pahalı"].RAM.median()),inplace=True)

Eksik verilerimizi doldurduk.

In [ ]:
df.isna().sum()

Var-Yok olarak yazılmış verilerimizi binary formatına dönüştürelim.

In [ ]:
df.replace("Yok",0,inplace=True)
df.replace("Var",1,inplace=True)

In [ ]:
df.head()

Renk özniteliğinin Fiyat Aralığı üzerindeki etkisini inceleyelim.

In [ ]:
trace1=go.Bar(
    x=df[df.FiyatAraligi=="Çok Ucuz"].Renk.value_counts().index,
    y=df[df.FiyatAraligi=="Çok Ucuz"].Renk.value_counts().values,
    name="Çok Ucuz",
    marker=dict(color="rgba(51, 102, 255,0.5)",
               line=dict(color="rgba(0,0,0)",width=1.5)
               ),
    )

trace2=go.Bar(
    x=df[df.FiyatAraligi=="Ucuz"].Renk.value_counts().index,
    y=df[df.FiyatAraligi=="Ucuz"].Renk.value_counts().values,
    name="Ucuz",
    marker=dict(color="rgba(0, 204, 102,0.5)",
               line=dict(color="rgba(0,0,0)",width=1.5)
               ),
    )

trace3=go.Bar(
    x=df[df.FiyatAraligi=="Normal"].Renk.value_counts().index,
    y=df[df.FiyatAraligi=="Normal"].Renk.value_counts().values,
    name="Normal",
    marker=dict(color="rgba(255, 255, 77,0.5)",
               line=dict(color="rgba(0,0,0)",width=1.5)
               ),
    )

trace4=go.Bar(
    x=df[df.FiyatAraligi=="Pahalı"].Renk.value_counts().index,
    y=df[df.FiyatAraligi=="Pahalı"].Renk.value_counts().values,
    name="Pahalı",
    marker=dict(color="rgba(255, 102, 153,0.5)",
               line=dict(color="rgba(0,0,0)",width=1.5)
               ),
    )


data=[trace1,trace2,trace3,trace4]
layout= go.Layout(barmode="group")

fig=go.Figure(data=data,layout=layout)
iplot(fig)

Grafiği incelediğimizde 'Renk' özniteliğimizin Fiyat Aralığı üzerinde bir etkisi olmadığını görebiliriz. 'Renk' özniteliğini binary olarak yazıp öznitelik sayısını arttırmaktansa 'Renk' özniteliğini silmek daha yararlı olacaktır.

In [ ]:
df.drop(['Renk'], axis=1,inplace=True)
df.head()

Veri setimizi hazırladık.

Veri setimizin korelasyon ısı haritasını çıkartarak yorumlayalım. Fiyat Aralığına diğer özniteliklerin etkisini incelemek için Fiyat Aralığını sayısal olarak gösterelim

In [ ]:
dfMain=df.copy()
dfMain.replace("Çok Ucuz",0,inplace=True)
dfMain.replace("Ucuz",1,inplace=True)
dfMain.replace("Normal",2,inplace=True)
dfMain.replace("Pahalı",3,inplace=True)

In [ ]:
corr = dfMain.corr()
f,ax=plt.subplots(figsize=(25,15))
sns.heatmap(corr, annot=True, ax=ax)
plt.show()

* Isı haritasını incelersek Fiyat aralığı ile RAM arasında pozitif ve güçlü bir ilişki olduğunu söyleyebiliriz. RAM arttıkça Fiyat aralığı da artıyordur. Bunun sebebi RAM donanım parçasının maliyetli olmasından kaynaklı olabilir. Çözünürlük yüksekliği, Çözünürlük genişliği ve Batarya gücünün de Fiyat aralığı üzerinde çok güçlü olmasa da pozitif bir ilişkisi olduğunu söyleyebiliriz

* Aynı şekilde 4G - 3G , OnKameraMP - ArkaKameraMP , ÇözünürlükGenislik - ÇözünürlükKalınlık ikilileri arasında da güçlü ve pozitif bir ilişki vardır. Bu ilişkilerin nedeni daha çok benzer öznitelikler olmasından kaynaklıdır.

Bu ilişkileri görsellşetirerek daha iyi inceleyelim;

In [ ]:
sns.jointplot("FiyatAraligi", "RAM", data=dfMain,
                  kind="kde", space=0, color="g")
plt.show()

In [ ]:
fiyatAraliklari=[0,1,2,3]
ramOrtalamasi=[dfMain[dfMain.FiyatAraligi==0].RAM.mean(),dfMain[dfMain.FiyatAraligi==1].RAM.mean(),dfMain[dfMain.FiyatAraligi==2].RAM.mean(),dfMain[dfMain.FiyatAraligi==3].RAM.mean()]
fig = px.bar( x=fiyatAraliklari, y=ramOrtalamasi, height=400, color=ramOrtalamasi)
fig.show()

In [ ]:
liste=[]
for i in range(2000):
    liste.append(i)
data=[
    {
        "y":dfMain.RAM,
        "x":liste,
        "mode":"markers",
        "marker":{
            "color":dfMain.FiyatAraligi,
            "showscale":True
        },
        "text":dfMain.FiyatAraligi
    }
]
iplot(data)

Tahmin ettiğimiz gibi RAM arttıkça Fiyat aralığı da artmıştır. Aralarında güçlü ve pozitif bir ilişki vardır

In [ ]:
dfMain.head()

In [ ]:
batayaGucuOrtalamasi=[dfMain[dfMain.FiyatAraligi==0].BataryaGucu.mean(),dfMain[dfMain.FiyatAraligi==1].BataryaGucu.mean(),dfMain[dfMain.FiyatAraligi==2].BataryaGucu.mean(),dfMain[dfMain.FiyatAraligi==3].BataryaGucu.mean()]
fig = px.bar( x=fiyatAraliklari, y=batayaGucuOrtalamasi, height=400, color=batayaGucuOrtalamasi)
fig.show()

In [ ]:
cozunurlukYukseklikOrtalamasi=[dfMain[dfMain.FiyatAraligi==0].CozunurlukYükseklik.mean(),dfMain[dfMain.FiyatAraligi==1].CozunurlukYükseklik.mean(),dfMain[dfMain.FiyatAraligi==2].CozunurlukYükseklik.mean(),dfMain[dfMain.FiyatAraligi==3].CozunurlukYükseklik.mean()]
fig = px.bar( x=fiyatAraliklari, y=cozunurlukYukseklikOrtalamasi, height=400, color=cozunurlukYukseklikOrtalamasi)
fig.show()

In [ ]:
cozunurlukGenislikOrtalamasi=[dfMain[dfMain.FiyatAraligi==0].CozunurlukYükseklik.mean(),dfMain[dfMain.FiyatAraligi==1].CozunurlukYükseklik.mean(),dfMain[dfMain.FiyatAraligi==2].CozunurlukYükseklik.mean(),dfMain[dfMain.FiyatAraligi==3].CozunurlukYükseklik.mean()]
fig = px.bar( x=fiyatAraliklari, y=cozunurlukGenislikOrtalamasi, height=400, color=cozunurlukGenislikOrtalamasi)
fig.show()

Görülüdüğü gibi Batarya Gücü, Çözünürlük Yüksekliği ve Çözünürülük Genişliği ile Fiyat aralığı üzerinde pozitif bir ilişki vardir. Ancak RAM özniteliği kadar güçlü bir ilişki değildir.

Şimdi de ArkaKameraMP - OnKameraMP ve 3G - 4G ikililerini görselleştirerek aralarındaki ilişkiyi inceleyelim

In [ ]:
plt.figure(figsize=(15,5))
sns.regplot(x=dfMain.OnKameraMP, y=dfMain.ArkaKameraMP, color="g")
plt.title("ArkaKameraMP-OnKameraMP Regression Plot")
plt.show()

Görüldüğü gibi Ön Kamera MP ile Arka Kamera MP arasında pozitf bir ilişki vardır

In [ ]:
ikisideYok=dfMain[(dfMain['3G']==0)&(dfMain['4G']==0)].shape[0]
BiriVar=dfMain[(dfMain['3G']==0)&(dfMain['4G']==1)].shape[0]+dfMain[(dfMain['3G']==1)&(dfMain['4G']==0)].shape[0]
ikisideVar=dfMain[(dfMain['3G']==1)&(dfMain['4G']==1)].shape[0]
listeVarYok=[ikisideYok,BiriVar,ikisideVar]
labels=['3G ve 4G yok','3G ve 4G den biri var','3G ve 4G de bulunanlar']
trace1={
    "values":listeVarYok,
    "labels":labels,
    "name":"3G ve 4G Bulundurma Durumu",
    "hoverinfo":"label+percent+name",
    "hole":.3,#ortada boşluk bırakıt
    "type":"pie"
}
data=[trace1];
layout={
    "title":"3G ve 4G Bulundurma Durumu",
    "annotations":[
        {"font":{"size":20},
        "showarrow":False,
        "text":"",
        "x":0.20,
        "y":1
        },
    ]
};
fig=go.Figure(data=data,layout=layout)
iplot(fig)

Bu grafikten yararlanarak 3G nin ve 4G nin beraber bulunduğu gözlemlerin çoğunlukta olduğunu söyleyebiliriz. Bu da koreleasyonlarının yüksek olmasını sağlamaktadır.

Veri setimizin üzerinde makine öğrenmesine başlamadan önce ön adımları gerçekleştirelim.

Bağımlı ve bağımsız değişkenleri bölütleyelim.

In [ ]:
y = df['FiyatAraligi']
x = df.drop(['FiyatAraligi'], axis=1)

Bağımlı değişkenimizi görüntüleyelim.

In [ ]:
y

Bağımlı değişkenlerimizin dağılımını inceleyelim.

In [ ]:
sns.countplot(x="FiyatAraligi", data=df)
plt.show()

Görüldüğü üzere bağımlı değişkenimiz veri setine eşit bir şekilde dağılmıştır.

Bağımsız değişkenimizi görüntüleyelim.

In [ ]:
x

Bağımlı ve bağımsız değişkenler üzerinden eğitim ve test olmak üzere bölütleme yapalım. Test oranını 25% belirleyelim.

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)

x_train'i görüntüleyelim.

In [ ]:
x_train

x_test'i görüntüleyelim.

In [ ]:
x_test

y_train'i görüntüleyelim.

In [ ]:
y_train

y_test'i görüntüleyelim.;

In [ ]:
y_test

Fiyat Aralığını en doğru şekilde tahmin edebilmek için en başarılı classification algoritmasının hangisi olduğuna bulalım.

# Naive Bayes Classification

Naive Bayes için kullanacağımız kütüphaneyi import edelim ve modelimizi inşa edelim.

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
nb.fit(x_train,y_train)

x_test de bulunun tüm gözlemleri tahmin edip y_pred değişkenine atayalım.

In [ ]:
y_pred = nb.predict(x_test)

Accuracy değerine bakalım.

In [ ]:
accuracy_score(y_test, y_pred)

Modeli farklı gözlemlerle kombinleyerek tekrardan skorunu hesaplayalım ve ortalamalarını alalım.

In [ ]:
cross_val_score(nb, x_test, y_test, cv = 10).mean()

Yaklaşık olarak %80 doğruluk başarısı elde ediyoruz. Karmaşıklık matrisini çizdirerek ineleyelim.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

Karmaşıklık matrisini incelediğimizde gözlemlerin 406 tanesini doğru,94 tanesini ise yanlış tahmin ettiğini söyleyebiliriz.

Sınıflandırma raporuna bakalım.

In [ ]:
report=classification_report(y_test, y_pred)
print(report)

Modelin daha iyi accuracy score vermesi için feature selection yapalım.

In [ ]:
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

Tüm öznitelikleri ve skorlarını görüntüleyelim.

In [ ]:
test = SelectKBest(k = 11)
fit = test.fit(x, y)
for indis, skor in enumerate(fit.scores_):
    print(skor, " -> ", x.columns[indis])

Doğruluk skorunu yükseltmek istiyoruz, bu yüzden sadece en anlamlı olan öznitelikleri seçelim ve tekrardan model kurup eğitelim.

In [ ]:
y1 = df['FiyatAraligi']
x1 = df[["BataryaGucu", "4G", "DahiliBellek", "Kalinlik", "Agirlik", "CekirdekSayisi", "CozunurlukYükseklik","CozunurlukGenislik","RAM","Dokunmatik"]]

In [ ]:
x1_train, x1_test, y1_train, y1_test = train_test_split(x1,
                                                    y1, 
                                                    test_size = 0.25, 
                                                    random_state = 42)

In [ ]:
nb = GaussianNB()
nb.fit(x1_train, y1_train)
y1_pred = nb.predict(x1_test)
accuracy_score(y1_test, y1_pred)

Sınıflandırma raporuna tekrardan bakalım.

In [ ]:
reportNB=classification_report(y1_test, y1_pred)
print(reportNB)

In [ ]:
cross_val_score(nb, x1_test, y1_test, cv = 10).mean()

Feature selection yaparak accuracy score değerini yükselttik. değişimi classification report değerlerine bakarak da görebiliriz.

Navie Bayes Metodunu kullanarak %81 doğruluğunda bir başarı elde ettik.

# Desicion Tree Classifier

Descion Tree algoritmasında kullanacağımız kütüphaneyi import edelim ve modelimizi inşa edelim.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42, criterion='gini')
dtEntropy = DecisionTreeClassifier(random_state = 42, criterion='entropy')
dt_model = dt.fit(x_train, y_train)

Şimdilik parametre optimizasyonu yapmadan modeli eğitmiş olalım ve modelin karar ağacını görüntüleyelim

Karar Ağacını görüntüleyelim.

In [ ]:
from sklearn.tree.export import export_text
r = export_text(dt, feature_names = ['BataryaGucu', 'Bluetooth', 'MikroislemciHizi', 'CiftHat', 'OnKameraMP',
       '4G', 'DahiliBellek', 'Kalinlik', 'Agirlik', 'CekirdekSayisi',
       'ArkaKameraMP', 'CozunurlukYükseklik', 'CozunurlukGenislik', 'RAM',
       'BataryaOmru', '3G', 'Dokunmatik', 'WiFi'])

print(r)

Karar ağacını graphviz ile görselleştirelim.

In [ ]:
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG
from graphviz import Source
from IPython.display import display

graph = Source(tree.export_graphviz(dt, out_file = None, feature_names = x.columns, filled = True))
display(SVG(graph.pipe(format = 'svg')))

x_test gözlemlerini tahmin ettirelim ve y_pred'e atayalım. Doğruluk skorumuzu öğrenelim.

In [ ]:
y_pred = dt.predict(x_test)
accuracy_score(y_test, y_pred)

Karmaşıklık matrisini görüntüleyelim

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

Karmasiklik matrisini incelediğimizde 399 doğru, 91 yanlış tahmin olduğunu söyleyebiliriz.

Modeli farklı gözlemlerle kombinleyerek tekrardan skorunu hesaplayalım.

In [ ]:
cross_val_score(dt_model, x, y, cv = 10).mean()

Sınıflandırma raporuna bakalım.

In [ ]:
print(classification_report(y_test, y_pred))

%83,15 oranında doğruluk elde ettik. Doğruluk değerimizi arttırmak için feature selection uyguladığımız x1 ve y1 den yararlanarak accuracy score değerimizi görüntüleyelim.

In [ ]:
cross_val_score(dt_model, x1, y1, cv = 10).mean()

Desicion Tree modeli ile %83,5 Doğruluğuna ulaştık.

Desicion Tree modelimizi entropy paremetresini kullanarak tekrar uygulayalım ve sonuçları inceleyelim.

In [ ]:
clf_entropy = DecisionTreeClassifier( 
            criterion = "entropy", random_state = 100, 
            max_depth = 3, min_samples_leaf = 5) 
clf_entropy.fit(x1_train, y1_train) 

cross_val_score(dtEntropy, x1, y1, cv = 10).mean()



In [ ]:
reportDT=classification_report(y1_test, y1_pred)
print(reportDT)

Entropy paremetresini kullanarak daha iyi sonuç aldık.

### Random Forest

Random Forest Classification kullanarak Desicion Tree modelimizi geliştirelim ve en uygun tree sayısını bulalım.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=400,random_state=1,criterion = "entropy")
rf.fit(x1_train,y1_train)
rf.score(x1_test,y1_test)
y1_pred = rf.predict(x1_test)
cross_val_score(rf, x1, y1, cv = 10).mean()

In [ ]:
reportRF=classification_report(y1_test, y1_pred)
print(reportRF)

Random Tree Classification kullandığımızda %91,25 Doğruluk elde ettik.

# Knn Classification

Kütüphanelerimizi import edelim.

In [ ]:
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.neighbors import KNeighborsClassifier

Knn cllassification uygulayıp scorunu ekrana yazdıralım.

In [ ]:
knn= KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train,y_train)
prediction=knn.predict(x_test)
knn.score(x_test,y_test)

In [ ]:
cross_val_score(knn, x, y, cv = 10).mean()

En iyi n_neighbours değerini bulalım.

In [ ]:
knn_params = {"n_neighbors": np.arange(1,15)}

In [ ]:
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, knn_params, cv = 3)
knn_cv.fit(x_train, y_train)

In [ ]:
print("En iyi skor: " + str(knn_cv.best_score_))
print("En iyi parametreler: " + str(knn_cv.best_params_))

In [ ]:
score_list=[]
for each in range(1,15):
    knn2=KNeighborsClassifier(n_neighbors=each)
    knn2.fit(x_train,y_train)
    score_list.append(knn2.score(x_test,y_test))

plt.plot(range(1,15),score_list)
plt.xlabel("k values")
plt.ylabel("accuaracy")
plt.show()

En iyi n_neighbours paremetresini gözlemlemek için grafik çizdirelim

In [ ]:
knn= KNeighborsClassifier(n_neighbors = 11) # n_neighbors = K
knn.fit(x_train,y_train)
y_pred=knn.predict(x_test)
knn.score(x_test,y_test)

In [ ]:
cross_val_score(knn, x, y, cv = 10).mean()

Karmaşıklık matrisini görüntüleyelim.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

Karmasiklik matrisini incelediğimizde 496 doğru, 31 yanlış tahmin olduğunu söyleyebiliriz.

In [ ]:
reportKNN=classification_report(y_test, y_pred)
print(reportKNN)

Knn Algoritması ile 0,9315  oranında bir doğruluk bulduk.

### Sonuç

Tüm algoritmaların sonuçlarını sırasıyla inceleyelim; 

Naive Bayes kullanarak 0.81, Desicion Tree kullanarak 0.866, Random Forest kullanarak 0.9125, Knn kullanarak ise 0.9315 oranlarında doğruluk sonuçları aldık. 

Classification report değerlerine bakarak bizim için en iyi algoritmanın hangisi olduğuna karar verelim.

In [ ]:
print(reportNB)

In [ ]:
print(reportDT)

In [ ]:
print(reportRF)

In [ ]:
print(reportKNN)

Classification Report değerlerini incelediğimizde en iyi sonucu knn classification algoritmasının verdiğini söyleyebiliriz. Bu nedenle bizim için en uygun yöntem Knn Classsification olacaktır.